In [23]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.datasets import fetch_20newsgroups
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [64]:
pip install pyldavis

In [26]:
newsgroups_train = fetch_20newsgroups(subset='train',remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test',remove=('headers', 'footers', 'quotes'))

In [25]:
newsgroups_data=fetch_20newsgroups(subset='all',remove=('headers', 'footers', 'quotes'))

In [4]:
print(list(newsgroups_train.target_names))

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [27]:
print(list(newsgroups_data.target_names))

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [5]:
for i in list(newsgroups_train.target_names):
  print (i)

alt.atheism
comp.graphics
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.sys.mac.hardware
comp.windows.x
misc.forsale
rec.autos
rec.motorcycles
rec.sport.baseball
rec.sport.hockey
sci.crypt
sci.electronics
sci.med
sci.space
soc.religion.christian
talk.politics.guns
talk.politics.mideast
talk.politics.misc
talk.religion.misc


In [28]:
for i in list(newsgroups_data.target_names):
  print (i)

alt.atheism
comp.graphics
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.sys.mac.hardware
comp.windows.x
misc.forsale
rec.autos
rec.motorcycles
rec.sport.baseball
rec.sport.hockey
sci.crypt
sci.electronics
sci.med
sci.space
soc.religion.christian
talk.politics.guns
talk.politics.mideast
talk.politics.misc
talk.religion.misc


In [29]:
stemmer = SnowballStemmer("english")
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

In [30]:
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [31]:
processed_docs = []

for doc in newsgroups_data.data:
    processed_docs.append(preprocess(doc))

In [32]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [10]:
'''count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break'''

'count = 0\nfor k, v in dictionary.iteritems():\n    print(k, v)\n    count += 1\n    if count > 10:\n        break'

In [33]:
'''
OPTIONAL STEP
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n= 100000)

In [34]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [13]:
'''
Preview BOW for our sample preprocessed document
'''
'''document_num = 25
bow_doc_x = bow_corpus[document_num]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], 
                                                     dictionary[bow_doc_x[i][0]], 
                                                     bow_doc_x[i][1]))'''

'document_num = 25\nbow_doc_x = bow_corpus[document_num]\n\nfor i in range(len(bow_doc_x)):\n    print("Word {} ("{}") appears {} time.".format(bow_doc_x[i][0], \n                                                     dictionary[bow_doc_x[i][0]], \n                                                     bow_doc_x[i][1]))'

In [35]:
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 20, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

In [36]:
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.038*"game" + 0.021*"team" + 0.015*"player" + 0.015*"play" + 0.009*"season" + 0.007*"hockey" + 0.007*"leagu" + 0.006*"score" + 0.006*"lose" + 0.006*"basebal"


Topic: 1 
Words: 0.014*"govern" + 0.009*"encrypt" + 0.009*"state" + 0.008*"presid" + 0.007*"secur" + 0.007*"public" + 0.005*"clipper" + 0.004*"issu" + 0.004*"chip" + 0.004*"american"


Topic: 2 
Words: 0.017*"appl" + 0.015*"monitor" + 0.013*"print" + 0.013*"printer" + 0.011*"font" + 0.009*"qualiti" + 0.009*"speed" + 0.009*"better" + 0.007*"price" + 0.007*"simm"


Topic: 3 
Words: 0.019*"chip" + 0.016*"program" + 0.013*"softwar" + 0.012*"test" + 0.010*"sound" + 0.010*"machin" + 0.010*"data" + 0.009*"design" + 0.007*"hardwar" + 0.006*"differ"


Topic: 4 
Words: 0.017*"list" + 0.014*"mail" + 0.014*"inform" + 0.013*"avail" + 0.011*"includ" + 0.009*"send" + 0.008*"univers" + 0.008*"request" + 0.008*"contact" + 0.007*"archiv"


Topic: 5 
Words: 0.048*"armenian" + 0.017*"turkish" + 0.013*"muslim" + 0.012*"turkey" + 0.

In [68]:
num = 103
unseen_document = newsgroups_data.data[num]
print(unseen_document)


In the same way in which antisemite means anti-Jewish and not anti-all-
persons-of-who-are-semite, a "form of racism" means: A form of segregation
against all those who are different based on the religious identification.

AAP



In [69]:
# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.5275467038154602	 Topic: 0.017*"israel" + 0.016*"jew" + 0.013*"arab" + 0.013*"isra" + 0.010*"jewish"
Score: 0.4124532639980316	 Topic: 0.012*"believ" + 0.010*"exist" + 0.010*"mean" + 0.009*"christian" + 0.008*"reason"


In [44]:
print(newsgroups_test.target[num])

10


In [45]:
lda_model.get_document_topics(bow_vector)

[(17, 0.92083335)]

In [22]:
import pyLDAvis
movies_vis_data = pyLDAvis.prepare(**lda_model)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


TypeError: ignored

In [50]:
# Compute Perplexity
  # a measure of how good the model is. lower the better.
perplexity=lda_model.log_perplexity(bow_corpus)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

In [51]:
perplexity

-7.460426033789404

In [55]:
# Compute Coherence Score
from gensim.models import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()


In [56]:
coherence_lda

0.6497685229794832

In [67]:
# Visualize the topics
pyLDAvis.enable_notebook()
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
vis = gensimvis.prepare(lda_model, bow_corpus, dictionary)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.052145 -0.063801       1        1  10.071241
17    -0.187263 -0.034726       2        1  10.057177
13     0.091634 -0.161112       3        1   9.404225
6     -0.024212 -0.025511       4        1   7.322248
19     0.171984 -0.074227       5        1   7.220443
4     -0.130008  0.087610       6        1   4.975168
0      0.107807  0.110442       7        1   4.886176
9      0.126183 -0.088244       8        1   4.770871
14     0.004822  0.016350       9        1   4.727810
16     0.158556 -0.004829      10        1   4.436269
15    -0.249526  0.024754      11        1   4.254313
18     0.043748 -0.138822      12        1   3.714292
8      0.040107 -0.048841      13        1   3.635387
5      0.162796  0.031856      14        1   3.341819
12    -0.143163  0.004580      15        1   3.166063
11    -0.124936  0.160230      16        1   2.974399
2     -0.161923 -0.038459      17        1   2.910541
3     -0.144803 -0.088324      18        1   2.845535
7      0.075520 -0.045529      19        1   2.662624
10     0.130532  0.376604      20        1   2.623400, topic_info=         Term         Freq        Total Category  logprob  loglift
752    window  3578.000000  3578.000000  Default  30.0000  30.0000
953      file  4086.000000  4086.000000  Default  29.0000  29.0000
58   armenian  1803.000000  1803.000000  Default  28.0000  28.0000
9        game  2383.000000  2383.000000  Default  27.0000  27.0000
136     drive  2696.000000  2696.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
8       final   181.820497   897.565524  Topic20  -5.0929   2.0440
586      lead   169.855754   784.059593  Topic20  -5.1610   2.1112
9        game   173.356463  2383.828577  Topic20  -5.1406   1.0196
610     power   171.740866  2037.162058  Topic20  -5.1500   1.1674
126    second   165.905373  1656.518179  Topic20  -5.1845   1.3396

[1420 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
2187      2  0.203954  abbrevi
2187      7  0.067985  abbrevi
2187     15  0.679845  abbrevi
4436     10  0.093484    aberr
4436     12  0.841353    aberr
...     ...       ...      ...
1463     14  0.008547    young
1463     19  0.350409    young
1463     20  0.087602    young
5442     10  0.991843  zionism
2110     10  0.989630  zionist

[7294 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 18, 14, 7, 20, 5, 1, 10, 15, 17, 16, 19, 9, 6, 13, 12, 3, 4, 8, 11])

In [70]:
coherence_model_lda2 = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='u_mass')
coherence_lda2 = coherence_model_lda2.get_coherence()

In [71]:
coherence_lda2

-2.059901086764699

In [72]:
coherence_model_lda3 = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_npmi')
coherence_lda3 = coherence_model_lda3.get_coherence()

In [73]:
coherence_lda3

0.0933003888932773